<a href="https://colab.research.google.com/github/olesyamba/Risk_analysis/blob/main/Practice_task_Portfolio_VaR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача 1.

Зависимость цены актива $P$ от двух риск-факторов $X,\,Y$  моделируется функцией
$$
P(x,\,y) = \frac{x-y}{(1+y)^2}.
$$
   Факторы $X, Y$ нестационарны, но не обязательно положительны, так что имеет смысл переходить к разностям. Текущие значения риск-факторов: $X=0.05,\,y=0.07$. Найдите текущие чувствительности доходности актива к приращениям этих двух факторов.

In [ ]:
from autograd import grad

# Функция цены актива P
def price_asset(x, y):
  return (x - y) / (1 + y) ** 2

# Текущие значения факторов
x0, y0 = 0.05, 0.07

# Речь идет о ПРИРАЩЕНИЯХ, т.е. x1 = x0 + dx и y1 = y0 + dy

# Требуется определить чувствительность ДОХОДНОСТИ актика к ПРИРАЩЕНИЯМ; запишем функцию
def f(dx, dy):
  return price_asset(x0 + dx, y0 + dy) / price_asset(x0, y0) - 1

# Теперь определим чувствительности
print('Верный ответ:', grad(f, [0, 1])(0.0, 0.0),'\n')

print(f(0.001, 0.0)/0.001, f(0, 0.001)/0.001)

Верный ответ: (array(-50.), array(48.13084112)) 

-50.00000000000004 48.040131085113735


Задача 2

Приведённый код генерирует наблюдения за ценой актива и двумя риск-факторами. По этим данным оцените чувствительности доходностей актива к доходностям обоих риск-факторов.


In [ ]:
import numpy as np
import pandas as pd
np.random.seed(42)
y_data = np.cumprod(1+np.random.randn(100)/10).round(4)*100
x_data = (np.hstack([1, np.cumprod(1 + np.random.normal(0.1, 0.2, size=len(y_data) - 1) / 10 + 0.01 * np.diff(y_data))]) * 100).round(2)
p_data = (x_data * 100 / (x_data + y_data ** 2)).round(2) + np.random.normal(0, 0.1, size=len(x_data)).round(4)
df = pd.DataFrame({"P": p_data,"X": x_data,"Y": y_data},index=pd.period_range(end="2021-09-01",periods=len(p_data),freq="D"))
assert df.iloc[-1, 0] == 16.7613, "Something's wrong. Please contact the teacher."
df

,P,X,Y
2021-05-25,0.7857,100.00,104.97
2021-05-26,0.9258,96.72,103.52
2021-05-27,0.8961,103.35,110.22
2021-05-28,0.8483,121.03,127.01
2021-05-29,0.8554,116.69,124.03
...,...,...,...
2021-08-28,17.0357,101.47,22.26
2021-08-29,16.4507,103.93,22.92
2021-08-30,15.8800,103.76,23.52
2021-08-31,15.9907,105.13,23.53


In [ ]:
from statsmodels.formula.api import ols

# Вычисляем доходности цен и факторов
returns = df.pct_change().dropna()
display(returns)

# Строим множественную линейную регрессию
reg = ols("P ~ X + Y", returns).fit()
print(reg.summary())


,P,X,Y
2021-05-26,0.178312,-0.032800,-0.013813
2021-05-27,-0.032080,0.068548,0.064722
2021-05-28,-0.053342,0.171069,0.152332
2021-05-29,0.008370,-0.035859,-0.023463
2021-05-30,-0.260931,-0.022196,-0.023381
...,...,...,...
2021-08-28,0.275605,-0.024702,-0.146472
2021-08-29,-0.034340,0.024244,0.029650
2021-08-30,-0.034692,-0.001636,0.026178
2021-08-31,0.006971,0.013204,0.000425


                            OLS Regression Results                            
Dep. Variable:                      P   R-squared:                       0.736
Model:                            OLS   Adj. R-squared:                  0.731
Method:                 Least Squares   F-statistic:                     133.9
Date:                Thu, 03 Nov 2022   Prob (F-statistic):           1.69e-28
Time:                        12:19:09   Log-Likelihood:                 99.499
No. Observations:                  99   AIC:                            -193.0
Df Residuals:                      96   BIC:                            -185.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0176      0.009      1.888      0.0

In [ ]:
returns.corr()

,P,X,Y
P,1.000000,-0.436500,-0.806402
X,-0.436500,1.000000,0.772142
Y,-0.806402,0.772142,1.000000


Задача 3

История котировок акции дана ниже (исполните код). Других факторов риска нет. Опцион A на эту акцию имеет дельту, равную 0.7, а опцион Б имеет дельту -0.2. Используя исторический метод, оцените VaR на горизонте 1 день и на уровне 99% по портфелю, состоящему из 100 млн. в акциях, короткой позиции в 20 млн в опционах А и длинной позиции в 10 млн в опционах Б.



In [ ]:
!pip install yfinance
import yfinance as yf
quotes = yf.download("V", "2019-01-02", "2019-12-23")["Adj Close"]
quotes

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[*********************100%***********************]  1 of 1 completed


Date
2019-01-02    129.773071
2019-01-03    125.096519
2019-01-04    130.485809
2019-01-07    132.838760
2019-01-08    133.561264
                 ...    
2019-12-16    182.997513
2019-12-17    182.290054
2019-12-18    181.680801
2019-12-19    183.292282
2019-12-20    184.726852
Name: Adj Close, Length: 246, dtype: float64

In [ ]:
# Дельта - чувствительность доходности одного актива к доходности другого актива или риск-фактора

# Построим таблицу исторических сценариев доходностей акции (риск-фактора)
returns = quotes.pct_change().dropna()

# Вычисляем совокупную чувствительность портфеля
prt_delta = (100*1 + (- 20) * 0.7 + 10 * (-0.2) ) # обращаем внимание на короткую позицию, которая в стоимости портфеля учитывается с отрицательным знаком
print(prt_delta)

# Вычисляет исторический VaR по данным
print('VaR для акции:', - returns.quantile(1- .99, interpolation='higher'), '\n')
print('Верный ответ', - returns.quantile(1 -.99, interpolation='lower') * prt_delta,'\n')
# print('Верный ответ', - returns.quantile(.99, interpolation='lower') * (-prt_delta),'\n') #если портфельная дельта отрицательная

84.0
VaR для акции: 0.02864224169876306 

Верный ответ 2.651934794723885 

Верный ответ 2.1943976562139715 



Задача 4

Портфель состоит на 20% из акций А, на 50% -- из акций Б и на 30% -- из акций В. Для моделирования используется 3 риск-фактора: индекс S&P 500, цена нефти и курс евро к доллару. Чувствительности доходностей акций к доходностям риск-факторов даны в таблице.

|Акция|S&P|Нефть| Курс|
|:--|--:|--:|--:|
|А| 0.4 | -0.3 | 0 |
|Б| 0.3 | 0.5 | 0.3|
|В| 0.2 | 0 | -0.4|

Найдите VaR на уровне 99% по этому портфелю на горизонте 1 день, используя дельта-нормальный подход.



In [ ]:
quotes_SnP = yf.download("^GSPC", "2019-01-02", "2019-12-23")["Adj Close"]
quotes_Oil = yf.download("BZ=F", "2019-01-02", "2019-12-23")["Adj Close"]
quotes_FX = yf.download("EUR=X", "2019-01-02", "2019-12-23")["Adj Close"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
import pandas as pd
import scipy.stats as ss

# Составим таблицы (матрицы) весов портфеля и его чувствительностей
weights = pd.Series({'A': 0.2, 'B': 0.5, 'C': 0.3})
display(weights)
sensitivities = pd.DataFrame({'A': [0.4, -0.3, 0], 'B': [0.3, 0.5, 0.3], 'C': [0.2, 0, -0.4]}, index=['S&P', 'Oil', 'FX']).T
display(sensitivities)

factor_data = pd.concat([quotes_SnP, quotes_Oil, quotes_FX], axis=1)
factor_data.columns = ['S&P', 'Oil', 'FX']
factor_returns = factor_data.pct_change().dropna()#
display(factor_returns,'\n')

# Определим риск-параметры (кавариационная матрица факторов и вектор средних)
factor_covariance = factor_returns.cov()
factor_means = factor_returns.mean()
print('Ковариационная матрица факторов\n',factor_covariance, '\nВектор средних доходностней факторов \n', factor_means)

# Определяем волатильность доходности портфеля
delta = weights @ sensitivities
print('Deltas:\n',delta)

portfolio_sigma = np.sqrt(
    (weights @ sensitivities) @ factor_covariance @ (sensitivities.T @ weights)
)
portfolio_mu = (weights @ sensitivities) @ factor_means

VaR = -100*ss.norm(portfolio_mu, portfolio_sigma).ppf(0.01) # 0.99
print('Дельта-нормальный VaR 99%:', VaR)

A    0.2
B    0.5
C    0.3
dtype: float64

,S&P,Oil,FX
A,0.4,-0.3,0.0
B,0.3,0.5,0.3
C,0.2,0.0,-0.4


,S&P,Oil,FX
Date,,,
2019-01-03,-0.024757,0.018940,0.012688
2019-01-04,0.034336,0.019839,-0.006406
2019-01-07,0.007010,0.004732,-0.001697
2019-01-08,0.009695,0.024246,-0.006036
2019-01-09,0.004098,0.046321,0.002066
...,...,...,...
2019-12-16,0.007148,0.001840,0.005537
2019-12-17,0.000335,0.011631,-0.001124
2019-12-18,-0.000432,0.001059,-0.001292


'\n'

Ковариационная матрица факторов
           S&P       Oil        FX
S&P  0.000061  0.000043 -0.000005
Oil  0.000043  0.000386 -0.000004
FX  -0.000005 -0.000004  0.000010 
Вектор средних доходностней факторов 
 S&P    0.001021
Oil    0.000929
FX     0.000123
dtype: float64
Deltas:
 S&P    0.29
Oil    0.19
FX     0.03
dtype: float64
Дельта-нормальный VaR 99%: 1.086021278235305


Задача 5

Стоимость бескупонной облигации, обещающей выплату 1 денежной единицы через s лет от текущего момента t, зависит от ставки дисконтирования rt соответствующей срочности следующим образом:

$B_t = e^{-r_t(s)s}$

Дана ежедневная история ставок дисконтирования за последний год (первое значение соответствует самому старому наблюдению, последнее -- самому новому; единицы измерения %/годовых):

6.37,6.36,6.35,6.34,6.34,6.35,6.36,6.36,6.37,6.33,6.32,6.32,6.33,6.15,6.16,6.09,6.1,6.06,5.94,5.91,5.92,5.87,5.8,5.73,5.71,5.64,5.6,5.58,5.58,5.56,5.52,5.46,5.43,5.39,5.36,5.36,5.37,5.38,5.37,5.36,5.36,5.35,5.38,5.38,5.33,5.29,5.26,5.22,5.13,5.11,5.08,5.05,5,4.98,4.96,4.95,4.95,4.95,4.92,4.89,4.88,4.86,4.86,4.83,4.83,4.84,4.85,4.86,4.87,4.9,4.93,4.94,4.95,4.96,4.95,4.81,4.8,4.79,4.78,4.73,4.72,4.74,4.66,4.64,4.62,4.62,4.63,4.61,4.58,4.59,4.57,4.58,4.58,4.55,4.54,4.51,4.51,4.51,4.52,4.51,4.43,4.35,4.36,4.32,4.27,4.21,4.19,4.18,4.19,4.2,4.2,4.21,4.21,4.21,4.22,4.2,4.2,4.2,4.2,4.21,4.23,4.21,4.22,4.21,4.21,4.21,4.21,4.21,4.21,4.21,4.21,4.22,4.23,4.23,4.23,4.23,4.23,4.22,4.21,4.21,4.2,4.19,4.2,4.21,4.21,4.21,4.2,4.2,4.2,4.18,4.16,4.16,4.16,4.17,4.17,4.17,4.15,4.13,4.13,4.13,4.12,4.11,4.11,4.11,4.12,4.13,4.13,4.14,4.15,4.15,4.15,4.16,4.16,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.16,4.16,4.17,4.18,4.16,4.17,4.17,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.17,4.17,4.18,4.18,4.18,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.16,4.16,4.16,4.15,4.15,4.14,4.14,4.13,4.14,4.15,4.15,4.14,4.14,4.14,4.14,4.14,4.14,4.14,4.14,4.14,4.15,4.15,4.14,4.15,4.15,4.15,4.14,4.14,4.14,4.13,4.13,4.13,4.13,4.13,4.14,4.14,4.15,4.15,4.13,4.15,4.16,4.15,4.16,4.17,4.17,4.17,4.18,4.18,4.18,4.19,4.21,4.21,4.23,4.26,4.27,4.28,4.29,4.3,4.33,4.37,4.37,4.37,4.38,4.39,4.44,4.47,4.39,4.45,4.67,4.83,4.88,4.94,5.01,5.07,5.06,5.18,5.23,5.24,5.24,5.26,5.28,5.29,5.29,5.3,5.32,5.3,5.31,5.32,5.36,5.36,5.38,5.39,5.42,5.48,5.48,5.51,5.53,5.53,5.53,5.52,5.51,5.49,5.54,5.67,5.69,5.72,5.7,5.7,5.77,5.81,5.82,5.83,5.83,5.84,5.85,5.83,5.83,5.86,5.87,5.84,5.88,5.96,5.97,6.08,6.21,6.11,6.23,6.39,6.35,6.36,6.02,5.99,5.85,5.83,5.84,5.86,5.85,5.85,5.84,5.84,5.84,5.85,5.85,5.86,5.88,5.87,5.87,5.88,5.89,5.89,5.93,6.02,6.03,6.04,6.04,6.05,6.04,6.04,6.04,6.04,6.04,6.05,6.05,6.06,6.07,6.07,6.08,6.09,6.1,6.09,6.09,6.1,6.09

Известно, что портфель таких облигаций имеет текущую стоимость 300 млн и срок до погашения 1.7 лет.

Используя исторический метод, найдите VaR по этому портфелю на уровне 99% на горизонте 1 день.

In [ ]:
# Дюрация - чувствительность рыночной доходности облигации к приращению процентной ставки: dP/P = - D*dr
# По заданию нужно посчитать quantile(dP) = quantile(-D*P*dr)

# Подготавливаем данные
rates_data = [6.37,6.36,6.35,6.34,6.34,6.35,6.36,6.36,6.37,6.33,6.32,6.32,6.33,6.15,6.16,6.09,6.1,6.06,5.94,5.91,5.92,5.87,5.8,5.73,5.71,5.64,5.6,5.58,5.58,5.56,5.52,5.46,5.43,5.39,5.36,5.36,5.37,5.38,5.37,5.36,5.36,5.35,5.38,5.38,5.33,5.29,5.26,5.22,5.13,5.11,5.08,5.05,5,4.98,4.96,4.95,4.95,4.95,4.92,4.89,4.88,4.86,4.86,4.83,4.83,4.84,4.85,4.86,4.87,4.9,4.93,4.94,4.95,4.96,4.95,4.81,4.8,4.79,4.78,4.73,4.72,4.74,4.66,4.64,4.62,4.62,4.63,4.61,4.58,4.59,4.57,4.58,4.58,4.55,4.54,4.51,4.51,4.51,4.52,4.51,4.43,4.35,4.36,4.32,4.27,4.21,4.19,4.18,4.19,4.2,4.2,4.21,4.21,4.21,4.22,4.2,4.2,4.2,4.2,4.21,4.23,4.21,4.22,4.21,4.21,4.21,4.21,4.21,4.21,4.21,4.21,4.22,4.23,4.23,4.23,4.23,4.23,4.22,4.21,4.21,4.2,4.19,4.2,4.21,4.21,4.21,4.2,4.2,4.2,4.18,4.16,4.16,4.16,4.17,4.17,4.17,4.15,4.13,4.13,4.13,4.12,4.11,4.11,4.11,4.12,4.13,4.13,4.14,4.15,4.15,4.15,4.16,4.16,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.16,4.16,4.17,4.18,4.16,4.17,4.17,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.18,4.17,4.17,4.18,4.18,4.18,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.17,4.16,4.16,4.16,4.15,4.15,4.14,4.14,4.13,4.14,4.15,4.15,4.14,4.14,4.14,4.14,4.14,4.14,4.14,4.14,4.14,4.15,4.15,4.14,4.15,4.15,4.15,4.14,4.14,4.14,4.13,4.13,4.13,4.13,4.13,4.14,4.14,4.15,4.15,4.13,4.15,4.16,4.15,4.16,4.17,4.17,4.17,4.18,4.18,4.18,4.19,4.21,4.21,4.23,4.26,4.27,4.28,4.29,4.3,4.33,4.37,4.37,4.37,4.38,4.39,4.44,4.47,4.39,4.45,4.67,4.83,4.88,4.94,5.01,5.07,5.06,5.18,5.23,5.24,5.24,5.26,5.28,5.29,5.29,5.3,5.32,5.3,5.31,5.32,5.36,5.36,5.38,5.39,5.42,5.48,5.48,5.51,5.53,5.53,5.53,5.52,5.51,5.49,5.54,5.67,5.69,5.72,5.7,5.7,5.77,5.81,5.82,5.83,5.83,5.84,5.85,5.83,5.83,5.86,5.87,5.84,5.88,5.96,5.97,6.08,6.21,6.11,6.23,6.39,6.35,6.36,6.02,5.99,5.85,5.83,5.84,5.86,5.85,5.85,5.84,5.84,5.84,5.85,5.85,5.86,5.88,5.87,5.87,5.88,5.89,5.89,5.93,6.02,6.03,6.04,6.04,6.05,6.04,6.04,6.04,6.04,6.04,6.05,6.05,6.06,6.07,6.07,6.08,6.09,6.1,6.09,6.09,6.1,6.09]
rates_index = pd.period_range(end="2021-09-01", periods=len(rates_data), freq="D")
rates = pd.Series((rates_data), index=rates_index)
display(rates.diff())

dur = 1.7
q = .99
P = 300

display(-dur * P * rates.diff().dropna()/100)
print('VaR 99% =', (-dur * P * rates.diff().dropna()/100).quantile(1-q, interpolation='higher'))

2020-08-09     NaN
2020-08-10   -0.01
2020-08-11   -0.01
2020-08-12   -0.01
2020-08-13    0.00
              ... 
2021-08-28    0.01
2021-08-29   -0.01
2021-08-30    0.00
2021-08-31    0.01
2021-09-01   -0.01
Freq: D, Length: 389, dtype: float64

2020-08-10    0.051
2020-08-11    0.051
2020-08-12    0.051
2020-08-13   -0.000
2020-08-14   -0.051
              ...  
2021-08-28   -0.051
2021-08-29    0.051
2021-08-30   -0.000
2021-08-31   -0.051
2021-09-01    0.051
Freq: D, Length: 388, dtype: float64

VaR 99% = -0.6629999999999994
